In [ ]:
# Config
import os

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob

# Parallel processing
from joblib import Parallel, delayed

import re
from PIL import Image
import supervision as sv
from supervision.metrics import F1Score, MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO

In [ ]:
configs = ["obb_base_y8x", "obb_base_y11l", "obb_base_y11m", "obb_base", "obb_base_y8l", "obb_base_y8m"]
config_to_model_mapping = {}
for config in configs:
    with open(f"../configs/{config}.py") as f:
        config_content = f.read()
    model_name = re.search(r"model = \"([^']+?)\"", config_content).group(1)
    config_to_model_mapping[config] = model_name.replace(".pt", "")

print(config_to_model_mapping)

data_yml_save_dir = mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_save_path = join(data_yml_save_dir, "data.yml")
with open(data_yml_save_path, "w") as f:
    f.write(data_yml)

image_dir = "images" ## Put predict state image 
gt_dir = "labels" ## Put predict state labels
sv_dataset = sv.DetectionDataset.from_yolo(image_dir, gt_dir, data_yml_save_path)
len(sv_dataset)

In [ ]:
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"])

for config in configs:
    model_name = config_to_model_mapping[config]
    model = YOLO(f"../runs/obb/config_name={config}&train_folder=m0_obb_without_empty_train&val_folder=m0_obb_without_empty_val/weights/best.pt")
    results_dir = f"../runs/obb/config_name={config}&train_folder=m0_obb_without_empty_train&val_folder=m0_obb_without_empty_val&z_predict_region=m0_obb_without_empty_val"
    predictions_dir = join(results_dir, "labels")
    
    targets = []
    predictions = []
    for name, _, gt_detection in tqdm(sv_dataset):
        file_name = splitext(basename(name))[0]
        prediction_path = join(predictions_dir, f"{file_name}.txt")
        ultralytics_result = model(name, imgsz=640, iou=0.33, max_det=300, conf=0.001, verbose=False)[0]
        sv_detections = sv.Detections.from_ultralytics(ultralytics_result)
        targets.append(gt_detection)
        predictions.append(sv_detections)
    
    mAP_metric = MeanAveragePrecision(class_agnostic=False)
    mAP_result = mAP_metric.update(predictions, targets).compute()
    class_wise_result = mAP_result.ap_per_class[:, 0].tolist()
    
    mAP_metric = MeanAveragePrecision(class_agnostic=True)
    mAP_result = mAP_metric.update(predictions, targets).compute()
    class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
    result_df.loc[model_name, :] = class_wise_result + [mAP_result.mAP_scores[0].item()] + class_agnostic_result

In [ ]:
weights = pd.Series(np.concatenate([target.class_id for target in targets])).value_counts()
weight_sum = (1/weights).values.sum()
weight_mapping = {"CFCBK": 1/weights.get(0, 0), "FCBK": 1/weights.get(1, 0), "Zigzag": 1/weights.get(2, 0)}
weight_mapping, weight_sum

In [ ]:
result_df_ = result_df.copy()
result_df_.index.name = "Model"
result_df_ = result_df_.reset_index(drop=False).sort_values("mAP", ascending=True)
result_df_['Model'] = result_df_['Model'].apply(lambda x: f"\\texttt{{{x}}}")
result_df_['Weighted mAP'] = (result_df_['CFCBK'] * weight_mapping["CFCBK"] + result_df_['FCBK'] * weight_mapping["FCBK"] + result_df_['Zigzag'] * weight_mapping["Zigzag"]) / weight_sum
result_df_ = result_df_[["Model", "CFCBK", "FCBK", "Zigzag", "Weighted mAP"]]
result_df_ = result_df_.sort_values("Weighted mAP", ascending=True)
latex_code = result_df_.to_latex(formatters={"CFCBK": "{:.2f}".format, "FCBK": "{:.2f}".format, "Zigzag": "{:.2f}".format, "mAP": "{:.2f}".format, "Class-agnostic AP": "{:.2f}".format, "Weighted mAP": "{:.2f}".format}, index=False)
latex_code = latex_code.replace("l"*result_df_.shape[1], "l"+"r"*(result_df_.shape[1] - 1))
print(latex_code)